In [1]:
import time
import pandas as pd
import json
from kafka import KafkaConsumer


In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local') \
    .appName('Streaming') \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.4,io.delta:delta-spark_2.12:3.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()


25/08/06 18:35:34 WARN Utils: Your hostname, Sais-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.16.7.60 instead (on interface en0)
25/08/06 18:35:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/sainivasrangaraju/.ivy2/cache
The jars for the packages stored in: /Users/sainivasrangaraju/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a11b8b28-4aaa-4f8d-8522-8d1c4553372e;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/anaconda3/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.4 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.4 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 225ms :: artifacts dl 5ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	io.delta#

In [4]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DoubleType
from pyspark.sql.functions import col, from_json

In [5]:
df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "localhost:9094") \
            .option("subscribe", "earthquake-topic") \
            .option("startingOffsets", "earliest") \
            .load()

In [6]:
schema = StructType([
    StructField('id', dataType=StringType()),
    StructField('magnitude', dataType=DoubleType()),
    StructField('location', dataType=StringType()),
    StructField('time', dataType=IntegerType()),
    StructField('longitude', dataType=DoubleType()),
    StructField('latitude', dataType=DoubleType()),
    StructField('depth', dataType=DoubleType())
])

In [7]:
json_df = df.selectExpr('CAST(value AS STRING) as json_string') \
            .select(from_json('json_string', schema=schema).alias('data')) \
            .select('data.*')

In [ ]:
json_df.writeStream \
    .outputMode('append') \
    .option('path', 'data_collected/earthquake_delta_table') \
    .option('checkpointLocation', 'checkpointing/earthquake_checkpoint') \
    .format('delta') \
    .start()


25/08/06 18:35:38 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


25/08/06 18:35:38 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/08/06 18:35:39 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
